
# Description of the procedure that we have followed

First of all, to train our model we decided to use BipBip Haricot to concentrate our efforts on one class. We started from the Notebook that we saw during the lessons, and tried to enhance its performance through parameter tuning. After we realised that we couldn't do better than 0.25-0.30 in meanIoU, we changed our approach. Our idea was to use a more classic encoder-decoder approach, and we did it, but again our performance was not improving as much as we hoped.<br>
We then implemented the UNet that we saw at lesson, and after we managed to reach around 0.40-0.45 in meanIoU, we decided to insert an Attention Layer and then perform Transfer Learning. In order to do this, we substituted the encoder part of the UNet with a vgg-16 net, with non-trainable layers. However, we could not increase a lot the meanIoU, so we decided to implement tiling. <br>
Since we didn't want to lose too much time before training (which is time consuming by definition), after we implemented the function to divide the images in 256x256 patches, we generated all the ones that we needed for the training and the validation set. This means that the attached code has to be modified in order to create them at least at the first run. How to do it is explained in the notebook before the Data Augmentation cell.<br>
However, we realised that the UNet built with vgg-16 as encorder had too many parameters (around 13 millions) and by visualising our results we saw that the network was too biased towards the background class. So, we came back to the previous model, increasing the depth of our UNet. At the end, we managed to reach a meanIoU of 0.60-0.65.


# Preliminar Operations

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
import os
import tarfile

# os.environ["CUDA_VISIBLE_DEVICES"]="-1" 
import tensorflow as tf
import numpy as np

# Set the seed for random operations. 
# This let our experiments to be reproducible. 
SEED = 1234
tf.random.set_seed(SEED)  

In [ ]:
FREEZE_UNTIL_TL = 5
bs = 1
lr = 5e-6
patches = True

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
!unzip /content/drive/MyDrive/dataset/second_challenge.zip

In [ ]:
base_dir = '/content/Development_Dataset/Training/Bipbip/Haricot/'

In [ ]:
from os import listdir
from os.path import isfile, join
import random

def create_file_txt(path, percentage_validation):
  files = [f[:-4]  for f in listdir(path) if isfile(join(path, f)) and f.lower().endswith('.jpg')]
  files_validation = random.sample(files, int(len(files)*percentage_validation))
  for file in files_validation:
      files.remove(file)
  with open(os.path.join(path, "train.txt"), 'w') as f:
    for file in files:
        f.write(file+'\n')  
  with open(os.path.join(path, "val.txt"), 'w') as f:
    for file in files_validation:
        f.write(file+'\n') 
  
  return

create_file_txt(base_dir+'Images',0.2)

# Image Segmentation
## Build segmentation

In [ ]:
def get_patches_img_array(img):
    ksize_rows = 256
    ksize_cols = 256
    strides_rows = 256
    strides_cols = 256

    # The size of sliding window
    ksizes = [1, ksize_rows, ksize_cols, 1] 

    # How far the centers of 2 consecutive patches are in the image
    strides = [1, strides_rows, strides_cols, 1]

    # sample pixel consecutively
    rates = [1, 1, 1, 1] 

    # padding algorithm to used
    padding='VALID'

    patches = tf.image.extract_patches([img], ksizes, strides, rates, padding)

    return patches


In [ ]:
def rescale(arr):
    arr_min = arr.min()
    arr_max = arr.max()
    return (arr - arr_min) / (arr_max - arr_min)


In [ ]:
from PIL import Image

def create_images(path, dataset_dir, file):
    with open(file, 'r') as f:
      lines = f.readlines()
    
    subset_filenames = []
    for line in lines:
      subset_filenames.append(line.strip()) 
    
    
    for curr_filename in subset_filenames:
        print(curr_filename)
        img = Image.open(os.path.join(dataset_dir, 'Images', curr_filename + '.jpg'))
        mask = Image.open(os.path.join(dataset_dir, 'Masks', curr_filename + '.png'))
        img_arr = np.array(img)
        mask_arr = np.array(mask)


        img_patches = get_patches_img_array(img_arr)
        mask_patches = get_patches_img_array(mask_arr)
        for i in range(img_patches.shape[1]):
            for j in range(img_patches.shape[2]):
                mask_patch = mask_patches[0,i,j,]
                img_patch = img_patches[0,i,j,]
                
                mask_patch = tf.reshape(mask_patch, [256, 256, 3])
                img_patch = tf.reshape(img_patch, [256, 256, 3])

                name = curr_filename + "_"+str(i)+"_"+str(j)
                print("\t" + name)
                
                
                
                img_patch = img_patch.numpy()

                im = Image.fromarray((img_patch).astype(np.uint8),'RGB')
                
                mask_patch = mask_patch.numpy()
                msk = Image.fromarray(mask_patch.astype(np.uint8), 'RGB')
                
                im.save(os.path.join(path, name+".jpg" ))
                msk.save(os.path.join(path, name+".png" ))
                

In order to create patches, you will have to call twice the following function (one for the training set and one for the validation set). The parameters have to be initialised as you want, and are

*   The path where you want to save your patches
*   The directory in which the dataset is saved
*   The file txt that lists the training and validation files respectively

Remember to change in the class "CustomDatasetPatches" of the notebook the directory in which you put the patches (at the moment, we called them "/content/drive/MyDrive/ANN second challenge/kaggle/working/train" and "/content/drive/MyDrive/ANN second challenge/kaggle/working/valid")



In [ ]:
#create_images(path_to_save_patches, dataset_dir, file_txt_to_read)

In [ ]:
# ImageDataGenerator
# ------------------

from tensorflow.keras.preprocessing.image import ImageDataGenerator

apply_data_augmentation = True

# Create training ImageDataGenerator object
# We need two different generators for images and corresponding masks
if apply_data_augmentation:
    img_data_gen = ImageDataGenerator(rotation_range=10,
                                      width_shift_range=10,
                                      height_shift_range=10,
                                      zoom_range=0.3,
                                      horizontal_flip=True,
                                      vertical_flip=True,
                                      fill_mode='reflect')
    mask_data_gen = ImageDataGenerator(rotation_range=10,
                                       width_shift_range=10,
                                       height_shift_range=10,
                                       zoom_range=0.3,
                                       horizontal_flip=True,
                                       vertical_flip=True,
                                       fill_mode='reflect')

In [ ]:
from PIL import Image

class CustomDatasetPatches(tf.keras.utils.Sequence):

  """
    CustomDataset inheriting from tf.keras.utils.Sequence.

    3 main methods:
      - __init__: save dataset params like directory, filenames..
      - __len__: return the total number of samples in the dataset
      - __getitem__: return a sample from the dataset

    Note: 
      - the custom dataset return a single sample from the dataset. Then, we use 
        a tf.data.Dataset object to group samples into batches.
      - in this case we have a different structure of the dataset in memory. 
        We have all the images in the same folder and the training and validation splits
        are defined in text files.

  """

  def __init__(self, dataset_dir, which_subset, img_generator=None, mask_generator=None, 
               preprocessing_function=None, out_shape=[256, 256]):
    if which_subset == 'training':
      self.path = '/content/drive/MyDrive/ANN second challenge/kaggle/working/train'
    elif which_subset == 'validation':
      self.path = '/content/drive/MyDrive/ANN second challenge/kaggle/working/valid'
    else:
        raise ValueError

    
    
    self.subset_filenames = [f[:-4] for f in os.listdir(self.path) if f.endswith(".jpg")]
    
    
    self.which_subset = which_subset
    
    self.img_generator = img_generator
    self.mask_generator = mask_generator
    self.preprocessing_function = preprocessing_function
    
    
    
    
  def __len__(self):
        return len(self.subset_filenames)

  def __getitem__(self, index):
    # Read Image
    curr_filename = self.subset_filenames[index]
    
    img = Image.open(os.path.join(self.path, curr_filename + '.jpg'))
    mask = Image.open(os.path.join(self.path, curr_filename + '.png'))

    
    img_arr = np.array(img)
    mask_arr = np.array(mask)

    


    if self.which_subset == 'training':
      if self.img_generator is not None and self.mask_generator is not None:
        # Perform data augmentation
        # We can get a random transformation from the ImageDataGenerator using get_random_transform
        # and we can apply it to the image using apply_transform
        img_t = self.img_generator.get_random_transform(img_arr.shape, seed=SEED)
        mask_t = self.mask_generator.get_random_transform(mask_arr.shape, seed=SEED)
        img_arr = self.img_generator.apply_transform(img_arr, img_t)
        out_mask = np.zeros_like(mask_arr)
        for c in np.unique(mask_arr):
          if c > 0:
            curr_class_arr = np.float32(mask_arr == c)
            curr_class_arr = self.mask_generator.apply_transform(curr_class_arr, mask_t)
            # from [0, 1] to {0, 1}
            curr_class_arr = np.uint8(curr_class_arr)
            # recover original class
            curr_class_arr = curr_class_arr * c 
            out_mask += curr_class_arr
    
        mask_arr = out_mask
        
    new_mask_arr = np.zeros(mask_arr.shape[:2], dtype=mask_arr.dtype)
    
      # Use RGB dictionary in 'RGBtoTarget.txt' to convert RGB to target
    new_mask_arr[np.where(np.all(mask_arr == [216, 124, 18], axis=-1))] = 0
    new_mask_arr[np.where(np.all(mask_arr == [255, 255, 255], axis=-1))] = 1
    new_mask_arr[np.where(np.all(mask_arr == [216, 67, 82], axis=-1))] = 2
    
    mask_arr = new_mask_arr

    mask_arr = np.expand_dims(mask_arr, -1)    
    
    if self.preprocessing_function is not None:
        img_arr = self.preprocessing_function(img_arr)

    return img_arr, np.float32(mask_arr)

In [ ]:
from PIL import Image

class CustomDataset(tf.keras.utils.Sequence):

  """
    CustomDataset inheriting from tf.keras.utils.Sequence.

    3 main methods:
      - __init__: save dataset params like directory, filenames..
      - __len__: return the total number of samples in the dataset
      - __getitem__: return a sample from the dataset

    Note: 
      - the custom dataset return a single sample from the dataset. Then, we use 
        a tf.data.Dataset object to group samples into batches.
      - in this case we have a different structure of the dataset in memory. 
        We have all the images in the same folder and the training and validation splits
        are defined in text files.

  """

  def __init__(self, dataset_dir, which_subset, img_generator=None, mask_generator=None, 
               preprocessing_function=None, out_shape=[256, 256]):
    if which_subset == 'training':
      subset_file = os.path.join(dataset_dir, 'Images', 'train.txt')
    elif which_subset == 'validation':
      subset_file = os.path.join(dataset_dir, 'Images', 'val.txt')

    with open(subset_file, 'r') as f:
      lines = f.readlines()
    
    subset_filenames = []
    for line in lines:
      subset_filenames.append(line.strip()) 

    self.which_subset = which_subset
    self.dataset_dir = dataset_dir
    self.subset_filenames = subset_filenames
    self.img_generator = img_generator
    self.mask_generator = mask_generator
    self.preprocessing_function = preprocessing_function
    self.out_shape = out_shape

  def __len__(self):
    return len(self.subset_filenames)

  def __getitem__(self, index):
    # Read Image
    curr_filename = self.subset_filenames[index]
    img = Image.open(os.path.join(self.dataset_dir, 'Images', curr_filename + '.jpg'))
    mask = Image.open(os.path.join(self.dataset_dir, 'Masks', curr_filename + '.png'))

    # Resize image and mask
    img = img.resize(self.out_shape)
    mask = mask.resize(self.out_shape, resample=Image.NEAREST)
    
    img_arr = np.array(img)
    mask_arr = np.array(mask)

    


    if self.which_subset == 'training':
      if self.img_generator is not None and self.mask_generator is not None:
        # Perform data augmentation
        # We can get a random transformation from the ImageDataGenerator using get_random_transform
        # and we can apply it to the image using apply_transform
        img_t = self.img_generator.get_random_transform(img_arr.shape, seed=SEED)
        mask_t = self.mask_generator.get_random_transform(mask_arr.shape, seed=SEED)
        img_arr = self.img_generator.apply_transform(img_arr, img_t)
        out_mask = np.zeros_like(mask_arr)
        for c in np.unique(mask_arr):
          if c > 0:
            curr_class_arr = np.float32(mask_arr == c)
            curr_class_arr = self.mask_generator.apply_transform(curr_class_arr, mask_t)
            # from [0, 1] to {0, 1}
            curr_class_arr = np.uint8(curr_class_arr)
            # recover original class
            curr_class_arr = curr_class_arr * c 
            out_mask += curr_class_arr
    
        mask_arr = out_mask
        
    new_mask_arr = np.zeros(mask_arr.shape[:2], dtype=mask_arr.dtype)
    
      # Use RGB dictionary in 'RGBtoTarget.txt' to convert RGB to target
    new_mask_arr[np.where(np.all(mask_arr == [216, 124, 18], axis=-1))] = 0
    new_mask_arr[np.where(np.all(mask_arr == [255, 255, 255], axis=-1))] = 1
    new_mask_arr[np.where(np.all(mask_arr == [216, 67, 82], axis=-1))] = 2
    
    mask_arr = new_mask_arr

    mask_arr = np.expand_dims(mask_arr, -1)    
    
    if self.preprocessing_function is not None:
        img_arr = self.preprocessing_function(img_arr)

    return img_arr, np.float32(mask_arr)

In [ ]:
from tensorflow.keras.applications.vgg16 import preprocess_input

img_h = 256
img_w = 256

#if you are using vgg, add in the CustomDataset function "preprocessing_function=preprocess_input"
if patches :
    dataset = CustomDatasetPatches(base_dir, 'training', 
                            img_generator=img_data_gen, 
                            mask_generator=mask_data_gen, 
                            out_shape=[img_h,img_w]
                            )

    dataset_valid = CustomDatasetPatches(base_dir, 
                                  'validation', 
                                  out_shape=[img_h,img_w]
                            )


else :
    dataset = CustomDataset(base_dir, 'training', 
                            img_generator=img_data_gen, 
                            mask_generator=mask_data_gen, 
                            out_shape=[img_h,img_w]
                            )

    dataset_valid = CustomDataset(base_dir, 
                                  'validation', 
                                  out_shape=[img_h,img_w]
                                  )




In [ ]:

train_dataset = tf.data.Dataset.from_generator(lambda: dataset,
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([img_h, img_w, 3], [img_h, img_w,1]))

train_dataset = train_dataset.batch(bs)

train_dataset = train_dataset.repeat()

valid_dataset = tf.data.Dataset.from_generator(lambda: dataset_valid,
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([img_h, img_w, 3], [img_h, img_w,1]))
valid_dataset = valid_dataset.batch(bs)

valid_dataset = valid_dataset.repeat()

In [ ]:
import time
from matplotlib import cm
import matplotlib.pyplot as plt

%matplotlib inline

colors = {}
colors[0] = [0,0,0]
colors[1] = [255,124,18]
colors[2] = [255,255,255]
colors[3] = [216, 67, 82]



iterator = iter(train_dataset)

In [ ]:

fig, ax = plt.subplots(1, 2)

augmented_img, target = next(iterator)


augmented_img = augmented_img[0]   # First element

target = np.array(target[0,...,0])   # First element (squeezing channel dimension)



target_img = np.zeros([img_h, img_w, 3])


for i in range(0, 3):
    target_img[np.where(target == i)] = np.array(colors[i])[:3]

ax[0].imshow(np.uint8(augmented_img))
ax[1].imshow(np.uint8(target_img))




plt.show()


# Creating model

In [ ]:
def attention(query, value):

    # CNN layer.
    cnn_layer = tf.keras.layers.Conv1D(
        filters=100,
        kernel_size=4,
        # Use 'same' padding so outputs have the same shape as inputs.
        padding='same')
    
    # Query encoding of shape [batch_size, Tq, filters].
    query_seq_encoding = cnn_layer(query)
    #print(query_seq_encoding.shape)
    
    # Value encoding of shape [batch_size, Tv, filters].
    value_seq_encoding = cnn_layer(value)
    #print(value_seq_encoding.shape)
    
    
    # Query-value attention of shape [batch_size, Tq, filters].
    query_value_attention_seq = tf.keras.layers.Attention()(
        [query_seq_encoding, value_seq_encoding])

    return query_value_attention_seq

In [ ]:
def down_block(x, filters, kernel_size=(3, 3), padding="same", strides=1):
    c = tf.keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(x)
    c = tf.keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(c)
    p = tf.keras.layers.MaxPool2D((2, 2), (2, 2))(c)
    return c, p

def up_block(x, skip, filters, kernel_size=(3, 3), padding="same", strides=1):
    us = tf.keras.layers.UpSampling2D((2, 2))(x)
    att = attention(us, skip)
    concat = tf.keras.layers.Concatenate()([att, us])
    c = tf.keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(concat)
    c = tf.keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(c)
    return c

def bottleneck(x, filters, kernel_size=(3, 3), padding="same", strides=1):
    c = tf.keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(x)
    c = tf.keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(c)
    return c


In [ ]:
def UNetAtt(f = [16, 32, 64, 128, 256]):
    
    inputs = tf.keras.layers.Input((256, 256, 3))
    
    p0 = inputs
    
    # c is the output of the convolution block while p is the ouput of the pooling layer
    c1, p1 = down_block(p0, f[1]) # 256x256x3  -> 128x128x32
    c2, p2 = down_block(p1, f[2]) # 128x128x32 -> 64x64x64
    c3, p3 = down_block(p2, f[3]) # 64x64x64   -> 32x32x128
    c4, p4 = down_block(p3, f[4]) # 32x32x128  -> 16x16x256
    
    
    bn = bottleneck(p4, f[4]) # convolution layer
    
    u1 = up_block(bn, c4, f[3]) # 16x16x256  -> 32x32x128 
    u2 = up_block(u1, c3, f[2]) # 32x32x128  -> 64x64x64
    u3 = up_block(u2, c2, f[1]) # 64x64x64   -> 128x128x32
    u4 = up_block(u3, c1, f[0]) # 128x128x32 -> 256x256x16
    
    outputs = tf.keras.layers.Conv2D(3, (1, 1), padding="same", activation="sigmoid")(u4)
    model = tf.keras.models.Model(inputs, outputs)
    return model

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16

def UNetAttVGG():

    inputs = tf.keras.layers.Input(shape=(256, 256, 3), name="input_image")
    
    vgg = tf.keras.applications.VGG16(input_tensor=inputs, weights='imagenet', include_top=False, input_shape=(256, 256, 3))

    for layer in vgg.layers:
        layer.trainable = False 

    encoder_output = vgg.get_layer("block5_pool").output    


    
    # block1 : 256x256x3  -> 128x128x64
    # block2 : 128x128x64 -> 64x64x128
    # block3 : 64x64x128  -> 32x32x256
    # block4 : 32x32x256  -> 16x16x512
    # block5 : 16x16x512  -> 8x8x512

    
    bn = bottleneck(encoder_output, 512) # convolution

    
    u2 = up_block(bn, vgg.get_layer("block5_conv3").output, 512) 
    u3 = up_block(u2, vgg.get_layer("block4_conv3").output, 256) 
    u4 = up_block(u3, vgg.get_layer("block3_conv2").output, 128) 
    u5 = up_block(u4, vgg.get_layer("block2_conv2").output, 64) 
    u6 = up_block(u5, vgg.get_layer("block1_conv2").output, 32)

    outputs = tf.keras.layers.Conv2D(3, (1, 1), padding="same", activation="softmax")(u6)
    
    model = tf.keras.models.Model(inputs, outputs)

    return model

In [ ]:
model = UNetAtt()
model.summary()

## Prepare the model for training

In [ ]:
# Optimization params
# -------------------

# Loss
# Sparse Categorical Crossentropy to use integers (mask) instead of one-hot encoded labels
loss = tf.keras.losses.SparseCategoricalCrossentropy() 

optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
# -------------------

# Here we define the intersection over union for each class in the batch.
# Then we compute the final iou as the mean over classes
def meanIoU(y_true, y_pred):
    # get predicted class from softmax
    y_pred = tf.expand_dims(tf.argmax(y_pred, -1), -1)

    per_class_iou = []

    for i in range(1,3): # exclude the background class 0
      # Get prediction and target related to only a single class (i)
      class_pred = tf.cast(tf.where(y_pred == i, 1, 0), tf.float32)
      class_true = tf.cast(tf.where(y_true == i, 1, 0), tf.float32)
      intersection = tf.reduce_sum(class_true * class_pred)
      union = tf.reduce_sum(class_true) + tf.reduce_sum(class_pred) - intersection
    
      iou = (intersection + 1e-7) / (union + 1e-7)
      per_class_iou.append(iou)

    return tf.reduce_mean(per_class_iou)

# Validation metrics
# ------------------
metrics = [meanIoU]
# ------------------

# Compile Model
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

## Training

In [ ]:
import os
from datetime import datetime

cwd = os.getcwd()

exps_dir = os.path.join(cwd, 'drive/My Drive/Keras4/', 'multiclass_segmentation_experiments')
if not os.path.exists(exps_dir):
    os.makedirs(exps_dir)

now = datetime.now().strftime('%b%d_%H-%M-%S')

model_name = 'CNN'

exp_dir = os.path.join(exps_dir, model_name + '_' + str(now))
if not os.path.exists(exp_dir):
    os.makedirs(exp_dir)
    
callbacks = []

# Early Stopping
# --------------
early_stop = True
if early_stop:
    es_callback = tf.keras.callbacks.EarlyStopping( monitor='val_loss', 
                                                    patience=10, 
                                                   mode='min',
                                                  restore_best_weights = True)
    callbacks.append(es_callback)


model.fit(x=train_dataset,
          epochs=100,  #### set repeat in training dataset
          steps_per_epoch=len(dataset),
          validation_data=valid_dataset,
          validation_steps=len(dataset_valid),
          callbacks=callbacks)



In [ ]:
#de-comment if you wish to save the weights

#model.save_weights('/content/drive/My Drive/my_model')

# Generate submission

In [ ]:
import json

def read_rgb_mask(img_path):
    '''
    img_path: path to the mask file
    Returns the numpy array containing target values
    '''

    mask_img = Image.open(img_path)
    mask_arr = np.array(mask_img)

    new_mask_arr = np.zeros(mask_arr.shape[:2], dtype=mask_arr.dtype)

    # Use RGB dictionary in 'RGBtoTarget.txt' to convert RGB to target
    new_mask_arr[np.where(np.all(mask_arr == [216, 124, 18], axis=-1))] = 0
    new_mask_arr[np.where(np.all(mask_arr == [255, 255, 255], axis=-1))] = 1
    new_mask_arr[np.where(np.all(mask_arr == [216, 67, 82], axis=-1))] = 2

    return new_mask_arr

def rle_encode(img):
    '''
    img: numpy array, 1 - foreground, 0 - background
    Returns run length as string formatted
    '''
    pixels = img.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)


In [ ]:
import os
from tensorflow.keras.preprocessing import image

if patches :
    submission_dict = {} 
    test_path = '/content/Development_Dataset/Test_Dev'
    for dirs in os.listdir(test_path):
        dir = os.path.join(test_path, dirs)
        for subdir in os.listdir(dir):
            files_list = os.path.join(dir, subdir+'/Images')
            for file in os.listdir(files_list):
                print("Currently segmenting image " + file)
                if file.endswith('.jpg') or file.endswith('.png'):
                    img_name = file[:-4]
                else:
                    print("NOT AN IMAGE!")
                img_path = os.path.join(files_list, file)
                img = image.load_img(img_path, target_size=(1536, 2048))


                img_array = image.img_to_array(img)
                img_batch = np.expand_dims(img_array, axis=0)


                img_patches = get_patches_img_array(img_array)

                full_mask_arr = None

                for i in range(img_patches.shape[1]):

                    line_mask = None
                    for j in range(img_patches.shape[2]):

                        img_preprocessed = img_patches[0,i,j,]
                        img_preprocessed = tf.reshape(img_preprocessed, [256, 256, 3]).numpy().astype(np.uint8)

                        prediction = model.predict(x=tf.expand_dims(img_preprocessed,0))

                        mask_arr = np.array(prediction)
                        mask_arr = tf.argmax(mask_arr, -1)[0, ...] # (256, 256)

                        # horizontal stack -> axis = 1
                        if line_mask == None:
                            line_mask = mask_arr
                        else:
                            line_mask = tf.concat([line_mask, mask_arr], axis=1)

                    # vertical stack -> axis = 0
                    if full_mask_arr == None:
                        full_mask_arr = line_mask
                    else:
                        full_mask_arr = tf.concat([full_mask_arr, line_mask], axis=0)

                full_mask_arr = np.array(full_mask_arr)
                submission_dict[img_name] = {}
                submission_dict[img_name]['shape'] = full_mask_arr.shape
                submission_dict[img_name]['team'] = dirs
                submission_dict[img_name]['crop'] = subdir
                submission_dict[img_name]['segmentation'] = {}

                rle_encoded_crop = rle_encode(full_mask_arr == 1)
                rle_encoded_weed = rle_encode(full_mask_arr == 2)

                submission_dict[img_name]['segmentation']['crop'] = rle_encoded_crop
                submission_dict[img_name]['segmentation']['weed'] = rle_encoded_weed

    with open(os.path.join('/content/drive/My Drive', 'submission.json'), 'w') as f:
        json.dump(submission_dict, f)

else:
    submission_dict = {} 
    test_path = '/content/Development_Dataset/Test_Dev'
    for dirs in os.listdir(test_path):
      dir = os.path.join(test_path, dirs)
      for subdir in os.listdir(dir):
        files_list = os.path.join(dir, subdir+'/Images')
        for file in os.listdir(files_list):
          if file.endswith('.jpg') or file.endswith('.png'):
            img_name = file[:-4]
          else:
            print("NOT AN IMAGE!")

          img_path = os.path.join(files_list, file)
          img = image.load_img(img_path, target_size=(1536, 2048))
          img_array = image.img_to_array(img)
          img_batch = np.expand_dims(img_array, axis=0)
          img_preprocessed = img_batch

          prediction = model.predict(img_preprocessed)

          mask_arr = tf.argmax(prediction, -1)[0, ...] # (256, 256)
          mask_arr = np.array(mask_arr)

          submission_dict[img_name] = {}
          submission_dict[img_name]['shape'] = mask_arr.shape
          submission_dict[img_name]['team'] = dirs
          submission_dict[img_name]['crop'] = subdir
          submission_dict[img_name]['segmentation'] = {}

          rle_encoded_crop = rle_encode(mask_arr == 1)
          rle_encoded_weed = rle_encode(mask_arr == 2)

          submission_dict[img_name]['segmentation']['crop'] = rle_encoded_crop
          submission_dict[img_name]['segmentation']['weed'] = rle_encoded_weed

    with open(os.path.join('/content/drive/My Drive', 'submission.json'), 'w') as f:
        json.dump(submission_dict, f)

## Visualisation of the results

In [ ]:
import time
import matplotlib.pyplot as plt

from PIL import Image

%matplotlib inline

iterator = iter(valid_dataset)

In [ ]:
complete_image = Image.new('RGB',(256*8, 256*6), (255,255,255))
for i in range(6):
    for j in range(8):
        image,_ = next(iterator)
        image = image[0].numpy()
        im = Image.fromarray((image).astype(np.uint8),'RGB')
        complete_image.paste(im,(256*j, 256*i))
    
complete_image.show()

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(8, 8))
fig.show()
ax[0].imshow(np.uint8(complete_image))


fig.canvas.draw()
time.sleep(1)

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(8, 8))
fig.show()
image, target = next(iterator)

colors = {}
colors[0] = [0,0,0]
colors[1] = [255,124,18]
colors[2] = [255,255,255]
colors[3] = [216, 67, 82]



image = image[0]
target = target[0, ..., 0]

out_sigmoid = model.predict(x=tf.expand_dims(image, 0))

# Get predicted class as the index corresponding to the maximum value in the vector probability
# predicted_class = tf.cast(out_sigmoid > score_th, tf.int32)
# predicted_class = predicted_class[0, ..., 0]
predicted_class = tf.argmax(out_sigmoid, -1)

#out_sigmoid.shape

predicted_class = predicted_class[0, ...]

# Assign colors (just for visualization)
target_img = np.zeros([target.shape[0], target.shape[1], 3])
prediction_img = np.zeros([target.shape[0], target.shape[1], 3])

target_img[np.where(target == 0)] = [0, 0, 0]


for i in range(0, 3):
  target_img[np.where(target == i)] = np.array(colors[i])[:3]

prediction_img[np.where(predicted_class == 0)] = [0, 0, 0]
for i in range(0, 3):
  prediction_img[np.where(predicted_class == i)] = np.array(colors[i])[:3] 

ax[0].imshow(np.uint8(image))
ax[1].imshow(np.uint8(target_img))
ax[2].imshow(np.uint8(prediction_img))

fig.canvas.draw()
time.sleep(1)